In [ ]:
from ultralytics import YOLO
import cv2
import os
from pathlib import Path
import json
import numpy as np
from datetime import datetime

# Path to your trained model
MODEL_PATH = "/content/drive/MyDrive/Convolve/model/stamp_signature_detection_55/stamp_signature_detector/weights/best.pt"

# Input folder containing test images
INPUT_FOLDER = "/content/drive/MyDrive/Convolve/data/train"

# Output folder for results
OUTPUT_FOLDER = "/content/drive/MyDrive/Convolve/data/train_results_55_stamp_sign"

# Inference parameters
CONFIDENCE_THRESHOLD = 0.25 # Minimum confidence for detection
IOU_THRESHOLD = 0.45  # NMS IoU threshold
IMAGE_SIZE = 736    # Input image size

# Class mapping - MUST MATCH YOUR TRAINED MODEL!
CLASS_NAMES = {
    0: 'signature',  # Class 0: Signature
    1: 'stamp'       # Class 1: Stamp
}

# Visualization settings for each class
CLASS_COLORS = {
    0: (0, 0, 255),    # Red for signatures (class 0)
    1: (0, 255, 0)     # Green for stamps (class 1)
}

BOX_THICKNESS = 2
TEXT_COLOR = (255, 255, 255)  # White text
TEXT_THICKNESS = 2
FONT_SCALE = 0.6

# Save options
SAVE_IMAGES = True  # Save images with drawn bounding boxes
SAVE_ANNOTATIONS = True  # Save detection annotations as JSON
SAVE_CROPPED_OBJECTS = True  # Save cropped regions for both stamps and signatures
SEPARATE_BY_CLASS = True  # Save cropped objects in separate folders by class

In [ ]:
def create_output_folders():
    """Creates organized output folder structure"""
    folders = {
        'annotated_images': os.path.join(OUTPUT_FOLDER, 'annotated_images'),
        'annotations': os.path.join(OUTPUT_FOLDER, 'annotations'),
        'no_detections': os.path.join(OUTPUT_FOLDER, 'no_detections'),
        'summary': OUTPUT_FOLDER
    }
    
    # Add folders for cropped objects
    if SAVE_CROPPED_OBJECTS:
        if SEPARATE_BY_CLASS:
            folders['cropped_stamps'] = os.path.join(OUTPUT_FOLDER, 'cropped_objects', 'stamps')
            folders['cropped_signatures'] = os.path.join(OUTPUT_FOLDER, 'cropped_objects', 'signatures')
        else:
            folders['cropped_objects'] = os.path.join(OUTPUT_FOLDER, 'cropped_objects')
    
    for folder in folders.values():
        os.makedirs(folder, exist_ok=True)
    
    return folders

def draw_obb_on_image(image, obb_points, label, confidence, color, thickness):
    """
    Draws oriented bounding box on image
    
    Args:
        image: numpy array of image
        obb_points: array of 4 corner points [[x1,y1], [x2,y2], [x3,y3], [x4,y4]]
        label: class label string
        confidence: detection confidence
        color: BGR color tuple
        thickness: line thickness
    """
    # Convert points to integer
    points = np.array(obb_points, dtype=np.int32)
    
    # Draw the rotated rectangle
    cv2.polylines(image, [points], isClosed=True, color=color, thickness=thickness)
    
    # Add label with confidence
    text = f"{label} {confidence:.2f}"
    
    # Get text size for background rectangle
    (text_width, text_height), baseline = cv2.getTextSize(
        text, cv2.FONT_HERSHEY_SIMPLEX, FONT_SCALE, TEXT_THICKNESS
    )
    
    # Draw background rectangle for text
    text_x, text_y = points[0]
    cv2.rectangle(
        image,
        (text_x, text_y - text_height - baseline - 5),
        (text_x + text_width, text_y),
        color,
        -1  # Filled rectangle
    )
    
    # Draw text
    cv2.putText(
        image,
        text,
        (text_x, text_y - baseline),
        cv2.FONT_HERSHEY_SIMPLEX,
        FONT_SCALE,
        TEXT_COLOR,
        TEXT_THICKNESS
    )
    
    return image

def get_cropped_region(image, obb_points):
    """
    Extracts and returns the rotated cropped region
    
    Args:
        image: numpy array of image
        obb_points: array of 4 corner points
    
    Returns:
        cropped image region
    """
    # Get the rotated bounding rectangle
    rect = cv2.minAreaRect(obb_points.astype(np.float32))
    box = cv2.boxPoints(rect)
    box = np.int32(box)
    
    # Get width and height of the rotated rectangle
    width = int(rect[1][0])
    height = int(rect[1][1])
    
    # Avoid zero dimensions
    if width == 0 or height == 0:
        return None
    
    # Get rotation matrix
    src_pts = box.astype("float32")
    dst_pts = np.array([
        [0, height-1],
        [0, 0],
        [width-1, 0],
        [width-1, height-1]
    ], dtype="float32")
    
    # Compute transformation matrix
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)
    
    # Warp the image
    warped = cv2.warpPerspective(image, M, (width, height))
    
    return warped



In [ ]:
def process_images_batch(model_path, input_folder, output_folders):
    """
    Process all images in a folder and detect stamps and signatures
    
    Args:
        model_path: path to trained YOLO model
        input_folder: folder containing test images
        output_folders: dictionary of output folder paths
    
    Returns:
        summary statistics dictionary
    """
    
    # Load the trained model
    print(f"Loading model from: {model_path}")
    model = YOLO(model_path)
    
    # Verify class names
    print(f"Model classes: {model.names}")
    
    # Get list of image files
    image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif']
    image_files = [
        f for f in os.listdir(input_folder)
        if os.path.splitext(f.lower())[1] in image_extensions
    ]
    
    if not image_files:
        print(f"No images found in {input_folder}")
        return None
    
    print(f"\nFound {len(image_files)} images to process")
    print("="*60)
    
    # Statistics tracking
    stats = {
        'total_images': len(image_files),
        'images_with_detections': 0,
        'images_without_detections': 0,
        'total_stamps_detected': 0,
        'total_signatures_detected': 0,
        'total_detections': 0,
        'processing_time': 0,
        'detections': [],
        'no_detection_files': [],
        'class_distribution': {
            'stamp': 0,
            'signature': 0
        }
    }
    
    start_time = datetime.now()
    
    # Process each image
    for idx, image_file in enumerate(image_files, 1):
        image_path = os.path.join(input_folder, image_file)
        image_name = Path(image_file).stem
        image_ext = Path(image_file).suffix
        
        print(f"\n[{idx}/{len(image_files)}] Processing: {image_file}")
        
        # Read original image for visualization
        original_image = cv2.imread(image_path)
        
        # Run inference
        results = model.predict(
            source=image_path,
            conf=CONFIDENCE_THRESHOLD,
            iou=IOU_THRESHOLD,
            imgsz=IMAGE_SIZE,
            verbose=False
        )
        
        # Get results for this image
        result = results[0]
        
        # Check if OBB (oriented bounding boxes) are detected
        if result.obb is not None and len(result.obb) > 0:
            
            # Count detections by class
            stamp_count = 0
            signature_count = 0
            
            # Prepare annotation data
            image_annotations = {
                'image_name': image_file,
                'image_path': image_path,
                'detections': {
                    'stamps': [],
                    'signatures': []
                }
            }
            
            # Process each detection
            for det_idx, (obb, conf, cls) in enumerate(zip(result.obb.xyxyxyxy, result.obb.conf, result.obb.cls)):
                # Get OBB points (4 corners)
                obb_points = obb.cpu().numpy()
                confidence = float(conf.cpu().numpy())
                class_id = int(cls.cpu().numpy())
                class_name = model.names[class_id]
                
                # Count by class
                if class_id == 0:  # Stamp
                    stamp_count += 1
                    detection_key = 'stamps'
                elif class_id == 1:  # Signature
                    signature_count += 1
                    detection_key = 'signatures'
                else:
                    continue  # Skip unknown classes
                
                # Save detection info
                detection_info = {
                    'detection_id': det_idx + 1,
                    'class': class_name,
                    'class_id': class_id,
                    'confidence': confidence,
                    'obb_points': obb_points.tolist()
                }
                image_annotations['detections'][detection_key].append(detection_info)
                
                # Get color for this class
                color = CLASS_COLORS.get(class_id, (255, 255, 255))
                
                # Draw bounding box on image
                original_image = draw_obb_on_image(
                    original_image,
                    obb_points,
                    class_name,
                    confidence,
                    color,
                    BOX_THICKNESS
                )
                
                # Save cropped object region
                if SAVE_CROPPED_OBJECTS:
                    cropped_object = get_cropped_region(cv2.imread(image_path), obb_points)
                    
                    if cropped_object is not None:
                        if SEPARATE_BY_CLASS:
                            if class_id == 0:
                                crop_folder = output_folders['cropped_stamps']
                                crop_filename = f"{image_name}_stamp_{stamp_count}{image_ext}"
                            else:
                                crop_folder = output_folders['cropped_signatures']
                                crop_filename = f"{image_name}_signature_{signature_count}{image_ext}"
                        else:
                            crop_folder = output_folders['cropped_objects']
                            crop_filename = f"{image_name}_{class_name}_{det_idx+1}{image_ext}"
                        
                        crop_path = os.path.join(crop_folder, crop_filename)
                        cv2.imwrite(crop_path, cropped_object)
            
            # Update statistics
            total_detected = stamp_count + signature_count
            if total_detected > 0:
                stats['images_with_detections'] += 1
                stats['total_stamps_detected'] += stamp_count
                stats['total_signatures_detected'] += signature_count
                stats['total_detections'] += total_detected
                stats['class_distribution']['stamp'] += stamp_count
                stats['class_distribution']['signature'] += signature_count
                
                print(f"  ✓ Found: {stamp_count} stamp(s), {signature_count} signature(s)")
                
                # Save annotated image
                if SAVE_IMAGES:
                    output_image_path = os.path.join(
                        output_folders['annotated_images'],
                        f"{image_name}_annotated{image_ext}"
                    )
                    cv2.imwrite(output_image_path, original_image)
                
                # Save annotations as JSON
                if SAVE_ANNOTATIONS:
                    annotation_path = os.path.join(
                        output_folders['annotations'],
                        f"{image_name}.json"
                    )
                    with open(annotation_path, 'w') as f:
                        json.dump(image_annotations, f, indent=2)
                
                stats['detections'].append(image_annotations)
            else:
                print(f"  ✗ No stamps or signatures detected")
                stats['images_without_detections'] += 1
                
                # Copy to no detections folder
                no_detection_path = os.path.join(output_folders['no_detections'], image_file)
                shutil.copy(image_path, no_detection_path)
                stats['no_detection_files'].append(image_file)
        else:
            print(f"  ✗ No detections")
            stats['images_without_detections'] += 1
            
            # Copy to no detections folder
            no_detection_path = os.path.join(output_folders['no_detections'], image_file)
            shutil.copy(image_path, no_detection_path)
            stats['no_detection_files'].append(image_file)
    
    # Calculate processing time
    end_time = datetime.now()
    stats['processing_time'] = (end_time - start_time).total_seconds()
    
    return stats

In [ ]:
def save_summary_report(stats, output_folder):
    """Saves a summary report of the inference results"""
    
    report_path = os.path.join(output_folder, 'detection_summary.json')
    
    summary = {
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'model_path': MODEL_PATH,
        'total_images_processed': stats['total_images'],
        'images_with_detections': stats['images_with_detections'],
        'images_without_detections': stats['images_without_detections'],
        'total_detections': stats['total_detections'],
        'stamps_detected': stats['total_stamps_detected'],
        'signatures_detected': stats['total_signatures_detected'],
        'class_distribution': stats['class_distribution'],
        'average_detections_per_image': stats['total_detections'] / stats['total_images'] if stats['total_images'] > 0 else 0,
        'processing_time_seconds': stats['processing_time'],
        'average_time_per_image': stats['processing_time'] / stats['total_images'] if stats['total_images'] > 0 else 0,
        'detections': stats['detections'],
        'no_detection_files': stats['no_detection_files']
    }
    
    with open(report_path, 'w') as f:
        json.dump(summary, f, indent=2)
    
    # Create comprehensive text summary
    text_report_path = os.path.join(output_folder, 'detection_summary.txt')
    with open(text_report_path, 'w') as f:
        f.write("="*60 + "\n")
        f.write("STAMP & SIGNATURE DETECTION SUMMARY REPORT\n")
        f.write("="*60 + "\n\n")
        f.write(f"Timestamp: {summary['timestamp']}\n")
        f.write(f"Model: {MODEL_PATH}\n\n")
        
        f.write("OVERALL STATISTICS\n")
        f.write("-" * 60 + "\n")
        f.write(f"Total images processed: {summary['total_images_processed']}\n")
        f.write(f"Images with detections: {summary['images_with_detections']}\n")
        f.write(f"Images without detections: {summary['images_without_detections']}\n")
        f.write(f"Detection rate: {(summary['images_with_detections']/summary['total_images_processed']*100):.1f}%\n\n")
        
        f.write("DETECTION BREAKDOWN\n")
        f.write("-" * 60 + "\n")
        f.write(f"Total detections: {summary['total_detections']}\n")
        f.write(f"  - Stamps (class 0): {summary['stamps_detected']}\n")
        f.write(f"  - Signatures (class 1): {summary['signatures_detected']}\n")
        f.write(f"Average detections per image: {summary['average_detections_per_image']:.2f}\n\n")
        
        f.write("PERFORMANCE\n")
        f.write("-" * 60 + "\n")
        f.write(f"Processing time: {summary['processing_time_seconds']:.2f} seconds\n")
        f.write(f"Average time per image: {summary['average_time_per_image']:.2f} seconds\n")
    
    # Create separate report for images without detections
    no_detections_report_path = os.path.join(output_folder, 'no_detections_list.txt')
    with open(no_detections_report_path, 'w') as f:
        f.write("="*60 + "\n")
        f.write("IMAGES WITHOUT DETECTIONS\n")
        f.write("="*60 + "\n\n")
        f.write(f"Total images without detections: {len(stats['no_detection_files'])}\n\n")
        f.write("List of files:\n")
        f.write("-" * 60 + "\n")
        for idx, filename in enumerate(stats['no_detection_files'], 1):
            f.write(f"{idx}. {filename}\n")
    
    # Create class-specific reports
    stamps_report_path = os.path.join(output_folder, 'stamps_detected_summary.txt')
    with open(stamps_report_path, 'w') as f:
        f.write("="*60 + "\n")
        f.write("STAMPS DETECTION SUMMARY (CLASS 0)\n")
        f.write("="*60 + "\n\n")
        f.write(f"Total stamps detected: {stats['total_stamps_detected']}\n")
        f.write(f"Images with stamps: {sum(1 for d in stats['detections'] if d['detections']['stamps'])}\n\n")
        f.write("Detections by image:\n")
        f.write("-" * 60 + "\n")
        for detection in stats['detections']:
            if detection['detections']['stamps']:
                f.write(f"{detection['image_name']}: {len(detection['detections']['stamps'])} stamp(s)\n")
    
    signatures_report_path = os.path.join(output_folder, 'signatures_detected_summary.txt')
    with open(signatures_report_path, 'w') as f:
        f.write("="*60 + "\n")
        f.write("SIGNATURES DETECTION SUMMARY (CLASS 1)\n")
        f.write("="*60 + "\n\n")
        f.write(f"Total signatures detected: {stats['total_signatures_detected']}\n")
        f.write(f"Images with signatures: {sum(1 for d in stats['detections'] if d['detections']['signatures'])}\n\n")
        f.write("Detections by image:\n")
        f.write("-" * 60 + "\n")
        for detection in stats['detections']:
            if detection['detections']['signatures']:
                f.write(f"{detection['image_name']}: {len(detection['detections']['signatures'])} signature(s)\n")
    
    return report_path, text_report_path, no_detections_report_path, stamps_report_path, signatures_report_path


In [ ]:
if __name__ == "__main__":
    
    print("\n" + "="*60)
    print("YOLOv8-OBB BATCH DETECTION")
    print("Stamps & Signatures Detection")
    print("="*60 + "\n")
    
    # Check if model exists
    if not os.path.exists(MODEL_PATH):
        print(f"Error: Model not found at {MODEL_PATH}")
        print("Please update MODEL_PATH to point to your trained model")
        exit(1)
    
    # Check if input folder exists
    if not os.path.exists(INPUT_FOLDER):
        print(f"Error: Input folder not found at {INPUT_FOLDER}")
        print("Please update INPUT_FOLDER to point to your test images")
        exit(1)
    
    # Create output folders
    print("Creating output folders...")
    output_folders = create_output_folders()
    
    # Process images
    print("\nStarting batch processing...")
    stats = process_images_batch(MODEL_PATH, INPUT_FOLDER, output_folders)
    
    if stats:
        # Save summary report
        print("\n" + "="*60)
        print("Processing Complete!")
        print("="*60)
        
        reports = save_summary_report(stats, OUTPUT_FOLDER)
        json_report, text_report, no_detections_report, stamps_report, signatures_report = reports
        
        print(f"\nResults Summary:")
        print(f"  Total images processed: {stats['total_images']}")
        print(f"  Images with detections: {stats['images_with_detections']}")
        print(f"  Images without detections: {stats['images_without_detections']}")
        print(f"  Total stamps detected: {stats['total_stamps_detected']}")
        print(f"  Total signatures detected: {stats['total_signatures_detected']}")
        print(f"  Processing time: {stats['processing_time']:.2f} seconds")
        
        print(f"\nOutput saved to: {OUTPUT_FOLDER}")
        print(f"  - Annotated images: {output_folders['annotated_images']}")
        if SEPARATE_BY_CLASS:
            print(f"  - Cropped stamps: {output_folders['cropped_stamps']}")
            print(f"  - Cropped signatures: {output_folders['cropped_signatures']}")
        else:
            print(f"  - Cropped objects: {output_folders['cropped_objects']}")
        print(f"  - JSON annotations: {output_folders['annotations']}")
        print(f"  - Images without detections: {output_folders['no_detections']}")
        print(f"  - Summary report: {json_report}")
        print(f"  - Text report: {text_report}")
        print(f"  - No detections list: {no_detections_report}")
        print(f"  - Stamps summary: {stamps_report}")
        print(f"  - Signatures summary: {signatures_report}")
        
        print("\n" + "="*60)